In [ ]:
"""
experiments.ipynb

File for running lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import copy
import functools
from importlib import reload
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

from src.harness import constants as C
from src.harness import dataset as ds
from src.harness import experiment
from src.harness import model as mod
from src.harness import pruning
from src.harness import rewind
from src.harness import training as train
from src.harness import utils

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Run Experiments

In [ ]:
reload(experiment)
reload(pruning)
reload(rewind)
reload(train)

# Test high level experiment API
experiment_summary: experiment.ExperimentSummary = experiment.run_experiments(
    5, 
    functools.partial(experiment.get_mnist_lenet_300_100_experiment_parameters, 0.2, 0.65),
    experiment.run_iterative_pruning_experiment,
)